<span style="float: left;padding: 1.3em">![logo](https://github.com/gw-odw/odw/blob/main/Tutorials/logo.png?raw=1)</span>

# Gravitational Wave Open Data Workshop

## Tutorial 3.3: Discovering and using published posterior samples

This is a simple demonstration to loading and viewing data released in associaton with the publication titled __GWTC-1: A Gravitational-Wave Transient Catalog of Compact Binary Mergers Observed by LIGO and Virgo during the First and Second Observing Runs__ available through [DCC](https://dcc.ligo.org/LIGO-P1800307/public) and [arXiv](https://arxiv.org/abs/1811.12907). This should lead to discussion and interpretation.

The data used in these tutorials will be downloaded from the public DCC page [LIGO-P1800370](https://dcc.ligo.org/LIGO-P1800370/public).

View this tutorial on [Google Colaboratory](https://colab.research.google.com/github/gw-odw/odw/blob/main/Tutorials/Day_3/Tuto_3.3_Discovering_and_using_published_posterior_samples.ipynb) or launch [mybinder](https://mybinder.org/v2/gh/gw-odw/odw/HEAD).

## Installation (execute only if running on a cloud platform, like Google Colab, or if you haven't done the installation already!)

> ⚠️ **Warning**: restart the runtime after running the cell below.
>
> To do so, click "Runtime" in the menu and choose "Restart and run all".

In [1]:
# -- Use the following line for google colab removing the hash at the beginning.
! pip install -U -q 'corner==2.2.3' 'bilby==2.4.0' 'astropy==7.0.1' 'scipy==1.12.0' 'numpy==1.25.2'

## Initialization

In [ ]:
import numpy as np
# The first import of matplotlib can take some time (especially on cloud platforms). This is normal.
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import corner

## Get the data

We first have to select the event. Let's pick GW150914!

In [ ]:
label = 'GW150914'

# if you do not have wget installed, simply download manually
# https://dcc.ligo.org/LIGO-P1800370/public/GW150914_GWTC-1.hdf5
# from your browser
# Learn more about this dataset here: https://dcc.ligo.org/LIGO-P1800370/public
! wget --no-verbose https://dcc.ligo.org/LIGO-P1800370/public/{label}_GWTC-1.hdf5

In [ ]:
#If you download the file manually, you may need to move the downloaded file to the correct foalder or substitute './' with the correct path
posterior_file = './'+label+'_GWTC-1.hdf5'
posterior = h5py.File(posterior_file, 'r')

### Looking into the file structure

In [ ]:
print('This file contains four datasets: ',posterior.keys())

This data file contains several datasets, two using separate models for the gravitational waveform (`IMRPhenomPv2` and `SEOBNRv3` respectively, see the [paper](https://arxiv.org/abs/1811.12907) for more details).

It also contains a joint dataset, combining equal numbers of samples from each individual model, these datasets are what is shown in the [paper](https://arxiv.org/abs/1811.12907).

Finally, there is a dataset containing samples drawn from the prior used for the analyses.

In [ ]:
print(posterior['Overall_posterior'].dtype.names)

Here are some brief descriptions of these parameters and their units:

 * `luminosity_distance_Mpc`: luminosity distance [Mpc]

 * `m1_detector_frame_Msun`: primary (larger) black hole mass (detector frame) [solar mass]

 * `m2_detector_frame_Msun`: secondary (smaller) black hole mass (detector frame) [solar mass]

 * `right_ascension`, `declination`: right ascension and declination of the source [rad].

 * `costheta_jn`: cosine of the angle between line of sight and total angular momentum vector of the system.

 * `spin1`, `costilt1`: primary (larger) black hole spin magnitude (dimensionless) and cosine of the zenith angle between the spin and the orbital angular momentum vector of the system.

 * `spin2`, `costilt2`: secondary (smaller) black hole spin magnitude (dimensionless) and cosine of the zenith angle between the spin and the orbital angular momentum vector of the system.

A convenient (and pretty) way to load up this array of samples is to use [pandas](https://pandas.pydata.org/):

In [ ]:
samples=pd.DataFrame.from_records(np.array(posterior['Overall_posterior']))

In [ ]:
samples

Those are all the samples stored in the `Overall` dataset.

### Plotting

We can plot all of them with, for instance, the [corner](https://corner.readthedocs.io/en/latest/) package:

In [ ]:
corner.corner(samples.values,labels=['costhetajn',
                                'distance [Mpc]',
                                'ra',
                                'dec',
                                'mass1 [Msun]',
                                'mass2 [Msun]',
                                'spin1',
                                'spin2',
                                'costilt1',
                                'costilt2']);

Each one and two dimensional histogram is a *marginalized* probability density function. We can manually select one parameter, say `luminosity distance`, and plot the four different marginalized distributions:

In [ ]:
plt.hist(posterior['prior']['luminosity_distance_Mpc'], bins = 100, label='prior', alpha=0.8, density=True)
plt.hist(posterior['IMRPhenomPv2_posterior']['luminosity_distance_Mpc'], bins = 100, label='IMRPhenomPv2 posterior', alpha=0.8, density=True)
plt.hist(posterior['SEOBNRv3_posterior']['luminosity_distance_Mpc'], bins = 100, label='SEOBNRv3 posterior', alpha=0.8, density=True)
plt.hist(posterior['Overall_posterior']['luminosity_distance_Mpc'], bins = 100, label='Overall posterior', alpha=0.8, density=True)
plt.xlabel(r'$D_L (Mpc)$')
plt.ylabel('Probability Density Function')
plt.legend()
plt.show()

### Computing new quantities

The masses given are the ones measured at the detector, i.e. in the *detector frame*. To get the actual (*source frame*) masses of the source black holes, we need to correct for the cosmological redshift of the gravitational wave. This forces us to assume a cosmological model:

In [ ]:
import astropy.units as u
from astropy.cosmology import Planck15, z_at_value

We now compute the redshift value for all the samples (using only the distance value). See [astropy.cosmology](https://docs.astropy.org/en/stable/api/astropy.cosmology.z_at_value.html) for implementation details, in particular how to make the following more efficient:

In [ ]:
z = np.array([z_at_value(Planck15.luminosity_distance, dist * u.Mpc) for dist in samples['luminosity_distance_Mpc']])

In [ ]:
samples['m1_source_frame_Msun']=samples['m1_detector_frame_Msun']/(1.0+z)
samples['m2_source_frame_Msun']=samples['m2_detector_frame_Msun']/(1.0+z)
samples['redshift']=z

And we can plot the marginalized probability density functions:

In [ ]:
corner.corner(samples[['m1_source_frame_Msun','m2_source_frame_Msun','redshift']].values,labels=['m1 (source)',
                                                                                          'm2 (source)',
                                                                                          'z']);

## Calculating credible intervals
Let's see how we can use Bilby to calculate summary statistics for the posterior like the median and 90% credible level.

In [ ]:
import bilby

# Make bilby more terse
bilby.core.utils.log.setup_logger(log_level='WARNING')

# calculate the detector frame chirp mass
mchirp = ((samples['m1_detector_frame_Msun'] * samples['m2_detector_frame_Msun'])**(3./5))/\
         (samples['m1_detector_frame_Msun'] + samples['m2_detector_frame_Msun'])**(1./5)
# initialize a SampleSummary object to describe the chirp mass posterior samples
chirp_mass_samples_summary = bilby.core.utils.SamplesSummary(samples=mchirp, average='median')
print('The median chirp mass = {} Msun'.format(chirp_mass_samples_summary.median))
print('The 90% credible interval for the chirp mass is {} - {} Msun'.format(chirp_mass_samples_summary.lower_absolute_credible_interval,
                                                                        chirp_mass_samples_summary.upper_absolute_credible_interval))

## Challenge question
Calculate the posterior for the effective spin, which is the mass-weighted component of the binary spin aligned to the orbital angular momentum. It is given by Eqn. 3 of https://journals.aps.org/prx/pdf/10.1103/PhysRevX.9.011001. The z-component of each component spin is defined as $\chi_{1z} = \chi_{1}\cos{\theta_{1}}$. Then initialize a `SamplesSummary` object for the chi_eff posterior and calculate the mean and the lower and upper absolute credible interval.